In [25]:
import nipype
import os,glob,sys,shutil
sys.path.append("/usr/lib/ants/")
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
import nipype.interfaces.ants.legacy as antsL
import nipype.interfaces.ants as ants

from nipype.caching import Memory
mem = Memory(base_dir='/data/HCP_Data/NipypeScratch/')
from IPython.display import Image
#!pip install --upgrade ipywidgets --user

In [7]:
# """
# Setup for DataGrabber inputs needed for the registration pipeline; This is using the freesurfer nodif and t1 masks
# """
# # """

ds = nio.DataGrabber(infields=['subject_id'],
    outfields=['nodif_brain','nodif_brain_mask','struct','struct_mask','struct_brain'])

datasource = pe.Node(interface=ds,name="datasource")
# create a node to obtain the functional images
datasource.inputs.base_directory = "/data/HCP_Data/HCP_BedpostData/"
datasource.inputs.template ='*'
datasource.inputs.sort_filelist = True
datasource.inputs.field_template = dict(
    nodif_brain='%s/T1w/Diffusion/nodif_brain.nii*',
    nodif_brain_mask='%s/T1w/Diffusion/nodif_brain_mask.nii*',
    struct='%s/T1w/T1w_acpc_dc.nii*',
    struct_mask='%s/T1w/brainmask_fs.nii*', 
    struct_brain='%s/T1w/T1w_acpc_dc_masked.nii*'
)

datasource.base_dir="/data/HCP_Data/NipypeScratch/datasource_cache"
datasource.inputs.template_args = dict(
             nodif_brain = [['subject_id']],
             nodif_brain_mask =  [['subject_id']],
             struct =  [['subject_id']],
             struct_mask = [['subject_id']],
             struct_brain = [['subject_id']]
        )

In [9]:
subjRootDir = "/data/HCP_Data/HCP_BedpostData/"
FULL_SUBJECT_LIST = [x for x in os.listdir(subjRootDir) if os.path.isdir( subjRootDir+x+'/T1w/Diffusion.bedpostX')]
print(len(FULL_SUBJECT_LIST),"Subjects are potentially available to be processed!")

711 Subjects are potentially available to be processed!


In [10]:
datasource.inputs.raise_on_empty = False

completeSubjs = []
missingSubjs = []
for s in FULL_SUBJECT_LIST:
    datasource.inputs.subject_id = s
    results = datasource.run()

    if None not in results.outputs.get().values():
        completeSubjs.append(s)
    else:
        missingSubjs.append(s)
print("Complete:",len(completeSubjs),"MISSING Scans:",len(missingSubjs))


180816-19:33:29,930 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:29,943 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:29,961 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:29,962 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:29,973 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:29,988 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:29,989 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:30,1 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:30,16 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:30,17 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/dat

180816-19:33:30,761 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:30,775 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:30,777 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:30,791 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:30,806 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:30,808 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:30,822 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:30,838 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:30,839 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:30,853 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber"

180816-19:33:31,628 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:31,644 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:31,660 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:31,662 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:31,676 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:31,692 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:31,693 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:31,708 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:31,724 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:31,726 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch

180816-19:33:32,528 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:32,543 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:32,545 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:32,561 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:32,576 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:32,578 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:32,594 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:32,610 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:32,611 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:32,628 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber"

180816-19:33:33,430 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:33,445 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:33,461 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:33,463 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:33,477 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:33,493 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:33,495 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:33,511 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:33,527 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:33,529 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch

...

180816-19:33:48,480 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:48,495 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:48,496 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:48,509 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:48,527 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:48,529 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:48,544 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:48,559 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:48,560 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:48,574 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber"

180816-19:33:49,361 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:49,376 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:49,392 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:49,394 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:49,409 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:49,427 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:49,429 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:49,443 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:49,459 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:49,460 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch

180816-19:33:50,259 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:50,275 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:50,277 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:50,292 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:50,307 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:50,310 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:50,324 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:50,340 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:50,342 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:50,357 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber"

180816-19:33:50,999 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:51,0 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:51,11 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:51,23 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:51,24 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:51,35 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:51,47 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:51,48 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:51,59 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:51,71 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:51,73 workflow INFO:
	 [Node] 

180816-19:33:51,715 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:51,731 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:51,733 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:51,748 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:51,763 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:51,765 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:51,779 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180816-19:33:51,795 workflow INFO:
	 [Node] Finished "datasource".
180816-19:33:51,797 workflow INFO:
	 [Node] Setting-up "datasource" in "/data/HCP_Data/NipypeScratch/datasource_cache/datasource".
180816-19:33:51,812 workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber"

In [13]:
"""
Setup for Registration  Pipeline InfoSource i.e. subjects
"""
subj_infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id']),  name="subj_infosource")
#infosource.iterables = ('subject_id', SampleSubjList)
subj_infosource.iterables = ('subject_id', completeSubjs)
### Above just converts the list of subjects into an iterable list I can connect to the next part of the pipeline

In [ ]:
#### RIGID BODY REGISTRATION OF DTI -- >  Struct Brain
from nipype.interfaces.ants import Registration

fsl_ExtractROI = mem.cache(fsl.ExtractROI)

reg_DTI_to_Struct = Registration()
reg_DTI_to_Struct.inputs.transforms = ['Rigid']  ## Mandatory
reg_DTI_to_Struct.inputs.dimension = 3
reg_DTI_to_Struct.inputs.float = False
reg_DTI_to_Struct.inputs.interpolation = 'NearestNeighbor'### or Linear?
reg_DTI_to_Struct.inputs.metric = ['MI']
reg_DTI_to_Struct.inputs.metric_weight = [1.0]
reg_DTI_to_Struct.inputs.smoothing_sigmas = [[3,2,1,0]]
reg_DTI_to_Struct.inputs.shrink_factors = [[8,4,2,1]]
reg_DTI_to_Struct.inputs.number_of_iterations= [[1000,500,250,100]]
reg_DTI_to_Struct.inputs.radius_or_number_of_bins=[32]
reg_DTI_to_Struct.inputs.radius_bins_stage_trait=[1]
reg_DTI_to_Struct.inputs.sampling_strategy = ['Regular']
reg_DTI_to_Struct.inputs.sigma_units=['vox']
reg_DTI_to_Struct.inputs.transform_parameters = [(0.1,)]
reg_DTI_to_Struct.inputs.winsorize_lower_quantile = 0.005
reg_DTI_to_Struct.inputs.winsorize_upper_quantile = 0.995
reg_DTI_to_Struct.inputs.use_histogram_matching = False
reg_DTI_to_Struct.inputs.num_threads = 20

In [ ]:
## This code is pretty well hyperthreaded, so not sure what advantage of running a full pipeline is..
regScratchDir = "/data/HCP_Data/NipypeScratch/run_hcp_reg/"


for subjID in completeSubjs:

    fixed = '/data/HCP_Data/HCP_BedpostData/%s/T1w/T1w_acpc_dc_masked.nii.gz' % subjID
    moving = '/data/HCP_Data/HCP_BedpostData/%s/T1w/Diffusion/nodif_brain.nii.gz' % subjID

    subjWD = os.path.join(regScratchDir,str(subjID))

    if not os.path.isdir(subjWD):
        os.makedirs(subjWD)


    #result = reg.run()
    reg_DTI_to_Struct.inputs.fixed_image = fixed
    reg_DTI_to_Struct.inputs.moving_image = moving
    reg_DTI_to_Struct.inputs.output_transform_prefix = os.path.join(regScratchDir,str(subjID),"%s_output_" % subjID)
    reg_DTI_to_Struct.inputs.output_warped_image = os.path.join(regScratchDir,str(subjID),"%s_DTI_to_Struct.nii.gz" % subjID)

    reg_DTI_to_Struct.run()

In [23]:
# wi = result.outputs.warped_image
# from niwidgets import NiftiWidget
# from niwidgets import examplet1

# test_widget = NiftiWidget(wi)
# test_widget.nifti_plotter()


# fixed_widget = NiftiWidget( "/data/HCP_Data/HCP_BedpostData/100206/T1w/T1w_acpc_dc_masked.nii.gz")
# fixed_widget.nifti_plotter()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=129, continuous_update=False, description='x', max=259), IntSlider(value…

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=129, continuous_update=False, description='x', max=259), IntSlider(value…

In [ ]:
fsl_ExtractROI = mem.cache(fsl.ExtractROI)
fsl_ImageMaths = mem.cache(fsl.ImageMaths)




In [ ]:
# antsL.antsIntroduction()
# ants.WarpImageMultiTransform()
run_hcp_reg  = pe.Workflow(name="run_hcp_reg")
run_hcp_reg.base_dir = "/data/HCP_Data/NipypeScratch/"

#samples_base_name_fxn = lambda x : x.replace('_th1samples.nii.gz','')

run_hcp_reg.connect(subj_infosource,'subject_id',datasource,'subject_id')
# ### Connect the dti_datasource to the pbx2 command
run_hcp_reg.connect( datasource,'struct_brain',regDTI_to_Struct,'reference_image')
run_hcp_reg.connect( datasource,'nodif_brain',regDTI_to_Struct,'input_image')
# runpbx2.connect( datasource,'fsamples',pbx2,'fsamples')
# runpbx2.connect( datasource,'nodif_brain_mask',pbx2,'mask')
# runpbx2.connect( datasource, ('thsamples', samples_base_name_fxn ), pbx2,'samples_base_name') ###  NOTE THIS IS A WEIRD TUPLE IS

#run_hcp_reg.run()
run_hcp_reg.run(plugin='MultiProc', plugin_args={'n_procs' : 10})

In [ ]:
### Concepts/Steps

## 3 Spaces     DTI    ---- >    T1        --->MNI
## DTI is probably "best" registered using the nodif_brain which is the first image of the "data.nii.gz" file


In [ ]:
# DTI='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/data.nii.gz'
# DTI_B0='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain.nii.gz'
# DTI_M='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain_mask.nii.gz'
# T1_M='/data/HCP_Data/HCP_BedpostData/100610/T1w/brainmask_fs.nii.gz'
# T1='/data/HCP_Data/HCP_BedpostData/100610/T1w/T1w_acpc_dc.nii.gz'
# # DTI_TO_T1='/data/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/nodif_to_T1_deformed.nii.gz
# MNI_BRAIN='/usr/share/fsl/5.0/data/standard/MNI152_T1_1mm_brain.nii.gz'
# #T1_BRAIN=''/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/T1w_acpc_dc_masked.nii.gz
# ROI_ONE='/data/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Left.nii.gz' 


In [ ]:

# echo $IN $T1_M $T1

# ## NOTES PROBABLY wouLD bE QUICKER IF WE USED THE T1 IMAGE WIHT THE MASK APPLIED... TO DO!!

# export PATH=$PATH:/usr/lib/ants
# export ANTSPATH=/usr/lib/ants

# ## Get the first B0 image from the DTI data set
# #fslroi $DTI $DTI_B0 0 1 


# ## Register the B0/DTI image to the T1 --- we perhaps should have used the masked/brainonly T1??
# #antsIntroduction.sh -d 3 -r $T1 -i $DTI_B0 -o nodif_to_T1_ -t RI


# ## Need to apply the brain mask to the T1 image to speed things up
# #fslmaths $T1 -mas $T1_M $T1_MASKED


# ### Thsi registers the T1 Brain image nonlinearly to MNI Space.. and also generated the inverse warp
# #antsIntroduction.sh -d 3 -r $MNI_BRAIN  -i $T1_BRAIN -o T1_to_MNI_ -t GR

# ### note T1_To_MNI_Warp is the warp transform.. and we add two affine matrices which are also real files
# #WarpImageMultiTransform 3 $DTI_B0  nodif_to_MNI_test.nii.gz -R $MNI_BRAIN  T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt nodif_to_T1_Affine.txt


# ### THIS GOES IN REVERSE
# WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI.nii.gz -R $DTI_B0 -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz
# WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI_NN.nii.gz -R $DTI_B0 --use-NN -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz

# ## --use-NN: Use Nearest Neighbor Interpolation. 
 

END PRODUCT:   We will create a directory in Diffusion or maybe ROIs and maybe a subdirectory called DTI
    
    so /{SubjID}/T1w/ROIs/DTI_Space/Human_L_Hypothalamus.nii
       /{SubjID}/xfms/<<PUT XFMS HERE>> 
        
        

    
    

In [ ]:

# #
# #Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz
# #Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space
# #WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz

 

# #Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.

 

# #Apply transformations to get thresholded, normalized tracts into MNI space

# #WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

# #Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomi


### Start with the MNI-space hypothalamus / basal forebrain ROIs à want to end up with tractography in all subjects in MNI space

 

Compute rigid body transform from diffusion space to T1

antsIntroduction.sh -d 3 -r T1.nii.gz -i nodif.nii.gz -o nodif_to_T1_ -t RI

 

Compute nonlinear warp from T1 to MNI

antsIntroduction.sh -d 3 -r MNI.nii.gz -i T1.nii.gz -o T1_to_MNI_ -t GR

(I think GR (greedy Syn) should be faster than SY (Syn) which is the default and apparently I’ve been using GR with good results so far)

 

Apply transformations so you can check that it all looks OK

WarpImageMultiTransform 3 nodif.nii.gz nodif_to_MNI.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz

 

Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space

WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz

 

Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.

 

Apply transformations to get thresholded, normalized tracts into MNI space

WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomise

 

In [ ]:
!pwd

In [ ]:

thisfolder=${PWD}/sampleOutput
sub=11111 

antsRegistration --dimensionality 3 --float 0 \
--output [$thisfolder/NN_DTI_to_Struct_${sub}_,$thisfolder/NN_DTI_to_Struct_${sub}_Warped.nii.gz] \
--interpolation NearestNeighbor \
--winsorize-image-intensities [0.005,0.995] \
--use-histogram-matching 0 \
--transform Rigid[0.1] \
--metric MI[$T1_BRAIN,$DTI_B0,1,32,Regular,0.25] \
--convergence [1000x500x250x100,1e-6,10] \
--shrink-factors 8x4x2x1 \
--smoothing-sigmas 3x2x1x0vox 

#reg.inputs.convergence_threshold = [1.e-6]
#reg.inputs.convergence_window_size = [10]
# reg.inputs.use_histogram_matching = False
# reg.inputs.winsorize_upper_quantile = 0.995
# reg.inputs.winsorize_lower_quantile = 0.005

# >>> reg.inputs.metric_weight = [1]*2 # Default (value ignored currently by ANTs)
# >>> reg.inputs.radius_or_number_of_bins = [32]*2
# >>> reg.inputs.sampling_strategy = ['Random', None]
# >>> reg.inputs.sampling_percentage = [0.05, None]
# >>> reg.inputs.convergence_window_size = [20]*2
# >>> reg.inputs.sigma_units = ['vox'] * 2
# >>> reg.inputs.use_estimate_learning_rate_once = [True, True]
# >>> reg.inputs.use_histogram_matching = [True, True] # This is the default
# >>> reg.inputs.output_warped_image = 'output_warped_image.nii.gz'
# >>> reg.inputs.write_composite_transform = True
# >>> reg.inputs.collapse_output_transforms = False
# >>> reg.inputs.initialize_transforms_per_stage = False

# >>> reg.cmdline
# 'antsRegistration --collapse-output-transforms 0 --dimensionality 3 --initial-moving-transform [ trans.mat, 0 ] --initialize-transforms-per-stage 0 --interpolation Linear --output [ output_, output_warped_image.nii.gz ] --transform Affine[ 2.0 ] --metric Mattes[ fixed1.nii, moving1.nii, 1, 32, Random, 0.05 ] --convergence [ 1500x200, 1e-



In [ ]:

# >>> import copy, pprint
# >>> from nipype.interfaces.ants import Registration
# >>> reg = Registration()
# >>> reg.inputs.fixed_image = 'fixed1.nii'
# >>> reg.inputs.moving_image = 'moving1.nii'
# >>> reg.inputs.output_transform_prefix = "output_"
# >>> reg.inputs.initial_moving_transform = 'trans.mat'
# >>> reg.inputs.transforms = ['Affine', 'SyN']
# >>> reg.inputs.transform_parameters = [(2.0,), (0.25, 3.0, 0.0)]
# >>> reg.inputs.number_of_iterations = [[1500, 200], [100, 50, 30]]
# >>> reg.inputs.dimension = 3
# >>> reg.inputs.write_composite_transform = True
# >>> reg.inputs.collapse_output_transforms = False
# >>> reg.inputs.initialize_transforms_per_stage = False
# >>> reg.inputs.metric = ['Mattes']*2
# >>> reg.inputs.metric_weight = [1]*2 # Default (value ignored currently by ANTs)
# >>> reg.inputs.radius_or_number_of_bins = [32]*2
# >>> reg.inputs.sampling_strategy = ['Random', None]
# >>> reg.inputs.sampling_percentage = [0.05, None]
# >>> reg.inputs.convergence_threshold = [1.e-8, 1.e-9]
# >>> reg.inputs.convergence_window_size = [20]*2
# >>> reg.inputs.smoothing_sigmas = [[1,0], [2,1,0]]
# >>> reg.inputs.sigma_units = ['vox'] * 2
# >>> reg.inputs.shrink_factors = [[2,1], [3,2,1]]
# >>> reg.inputs.use_estimate_learning_rate_once = [True, True]
# >>> reg.inputs.use_histogram_matching = [True, True] # This is the default
# >>> reg.inputs.output_warped_image = 'output_warped_image.nii.gz'
# >>> reg.cmdline
# 'antsRegistration --collapse-output-transforms 0 --dimensionality 3 --initial-moving-transform [ trans.mat, 0 ] --initialize-transforms-per-stage 0 --interpolation Linear --output [ output_, output_warped_image.nii.gz ] --transform Affine[ 2.0 ] --metric Mattes[ fixed1.nii, moving1.nii, 1, 32, Random, 0.05 ] --convergence [ 1500x200, 1e-08, 20 ] --smoothing-sigmas 1.0x0.0vox --shrink-factors 2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --transform SyN[ 0.25, 3.0, 0.0 ] --metric Mattes[ fixed1.nii, moving1.nii, 1, 32 ] --convergence [ 100x50x30, 1e-09, 20 ] --smoothing-sigmas 2.0x1.0x0.0vox --shrink-factors 3x2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --winsorize-image-intensities [ 0.0, 1.0 ]  --write-composite-transform 1'
# >>> reg.run()  
# seedMask = "/data/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Bilat.nii.gz"
# seedMaskList = glob.glob('/data/HCP_Data/EHECHT_ROIS/Hu*')
# debug_output = False

# if debug_output:
#     pbx2 = fsl.ProbTrackX2()
#     pbx2.inputs.thsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_th1samples.nii.gz'
#     pbx2.inputs.phsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_ph1samples.nii.gz'
#     pbx2.inputs.fsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_f1samples.nii.gz'
#     pbx2.inputs.mask = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/nodif_brain_mask.nii.gz'
#     pbx2.inputs.samples_base_name = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged'
# else:
#     pbx2 = pe.Node(interface=fsl.ProbTrackX2(), name='pbx2')

    
# pbx2.inputs.c_thresh = 0.2   # -c 0.2   F cutoff
# pbx2.inputs.n_steps = 2000   # -S 2000
# pbx2.inputs.step_length = 0.5 # --steplength=0.5
# pbx2.inputs.n_samples = 5000  # -P 5000
# pbx2.inputs.opd = True
# pbx2.inputs.loop_check = True
# # pbx2.inputs.fibst = True

# pbx2.iterables = ("seed", seedMaskList)

# # pbx2.inputs.onewaycondition = True # --onewaycondition
# pbx2.inputs.correct_path_distribution = True # -pd  i.e. distance correction
# #pbx2.inputs.lrtarget3= targetMask  ## This is a hack for now
# pbx2.inputs.seed = seedMask
# #pbx2.inputs.omatrix3 = True   # --omatrix3
# #pbx2.inputs.target3 = targetMask # --target3=$WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_GM.nii.gz "; 

# ### #FIBTHRESH APPEARS MISSING FROM INTERFACE???
# if debug_output:
#     print( pbx2.cmdline)
#     print("WE WILL BE PROCESSING THE FOLLOWING ROIS")
#     print([os.path.basename(x) for x in seedMaskList])
#     pbx2.inputs.os2t = True

In [ ]:
seedMask = "/data/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Bilat.nii.gz"
seedMaskList = glob.glob('/data/HCP_Data/EHECHT_ROIS/Hu*')
debug_output = False

if debug_output:
    pbx2 = fsl.ProbTrackX2()
    pbx2.inputs.thsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_th1samples.nii.gz'
    pbx2.inputs.phsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_ph1samples.nii.gz'
    pbx2.inputs.fsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_f1samples.nii.gz'
    pbx2.inputs.mask = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/nodif_brain_mask.nii.gz'
    pbx2.inputs.samples_base_name = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged'
else:
    pbx2 = pe.Node(interface=fsl.ProbTrackX2(), name='pbx2')

    
pbx2.inputs.c_thresh = 0.2   # -c 0.2   F cutoff
pbx2.inputs.n_steps = 2000   # -S 2000
pbx2.inputs.step_length = 0.5 # --steplength=0.5
pbx2.inputs.n_samples = 5000  # -P 5000
pbx2.inputs.opd = True
pbx2.inputs.loop_check = True
# pbx2.inputs.fibst = True

pbx2.iterables = ("seed", seedMaskList)

# pbx2.inputs.onewaycondition = True # --onewaycondition
pbx2.inputs.correct_path_distribution = True # -pd  i.e. distance correction
#pbx2.inputs.lrtarget3= targetMask  ## This is a hack for now
pbx2.inputs.seed = seedMask
#pbx2.inputs.omatrix3 = True   # --omatrix3
#pbx2.inputs.target3 = targetMask # --target3=$WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_GM.nii.gz "; 

### #FIBTHRESH APPEARS MISSING FROM INTERFACE???
if debug_output:
    print( pbx2.cmdline)
    print("WE WILL BE PROCESSING THE FOLLOWING ROIS")
    print([os.path.basename(x) for x in seedMaskList])
    pbx2.inputs.os2t = True

In [ ]:
#runpbx.connect(subj_infosource,'subject_id',datasource,'subject_id')
runpbx2  = pe.Workflow(name="runpbx2")
runpbx2.base_dir = "/data/HCP_Data/NipypeScratch/"

samples_base_name_fxn = lambda x : x.replace('_th1samples.nii.gz','')


runpbx2.connect(subj_infosource,'subject_id',datasource,'subject_id')
# ### Connect the dti_datasource to the pbx2 command
runpbx2.connect( datasource,'thsamples',pbx2,'thsamples')
runpbx2.connect( datasource,'phsamples',pbx2,'phsamples')
runpbx2.connect( datasource,'fsamples',pbx2,'fsamples')
runpbx2.connect( datasource,'nodif_brain_mask',pbx2,'mask')
runpbx2.connect( datasource, ('thsamples', samples_base_name_fxn ), pbx2,'samples_base_name') ###  NOTE THIS IS A WEIRD TUPLE IS

#runpbx2.run()
runpbx2.run(plugin='MultiProc', plugin_args={'n_procs' : 30})

In [ ]:
DTI=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/data.nii.gz
DTI_B0=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain.nii.gz
DTI_M=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain_mask.nii.gz
T1_M=/HCP_Data/HCP_BedpostData/100610/T1w/brainmask_fs.nii.gz
T1=/HCP_Data/HCP_BedpostData/100610/T1w/T1w_acpc_dc.nii.gz
DTI_TO_T1=/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/nodif_to_T1_deformed.nii.gz
MNI_BRAIN=/usr/share/fsl/5.0/data/standard/MNI152_T1_1mm_brain.nii.gz 

T1_BRAIN=/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/T1w_acpc_dc_masked.nii.gz

ROI_ONE=/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Left.nii.gz  


echo $IN $T1_M $T1

## NOTES PROBABLY wouLD bE QUICKER IF WE USED THE T1 IMAGE WIHT THE MASK APPLIED... TO DO!!

export PATH=$PATH:/usr/lib/ants
export ANTSPATH=/usr/lib/ants

## Get the first B0 image from the DTI data set
#fslroi $DTI $DTI_B0 0 1 


## Register the B0/DTI image to the T1 --- we perhaps should have used the masked/brainonly T1??
#antsIntroduction.sh -d 3 -r $T1 i $DTI_B0 -o nodif_to_T1_ -t RI


## Need to apply the brain mask to the T1 image to speed things up
#fslmaths $T1 -mas $T1_M $T1_MASKED


### Thsi registers the T1 Brain image nonlinearly to MNI Space.. and also generated the inverse warp
#antsIntroduction.sh -d 3 -r $MNI_BRAIN  -i $T1_BRAIN -o T1_to_MNI_ -t GR

### note T1_To_MNI_Warp is the warp transform.. and we add two affine matrices which are also real files
#WarpImageMultiTransform 3 $DTI_B0  nodif_to_MNI_test.nii.gz -R $MNI_BRAIN  T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt nodif_to_T1_Affine.txt


### THIS GOES IN REVERSE
WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI.nii.gz -R $DTI_B0 -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz
WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI_NN.nii.gz -R $DTI_B0 --use-NN -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz

## --use-NN: Use Nearest Neighbor Interpolation. 
 

#
#Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz
#Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space
#WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz

 

#Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.

 

#Apply transformations to get thresholded, normalized tracts into MNI space

#WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

#Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomi

